In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('/kaggle/input/crop-recommendation-dataset/Crop_recommendation.csv')

In [ ]:
df

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df_enc = df
df_enc['label'] = encoder.fit_transform(df['label']) 
corr_matrix = df_enc.corr()
plt.figure(figsize=(14,10))
sns.heatmap(corr_matrix,annot=True,cmap='Reds')

In [ ]:
X= df.drop(columns = ['label', 'ph', 'rainfall'])
y= df['label']

In [ ]:
X

In [ ]:
y

In [ ]:
num_classes = len(np.unique(y))
print(num_classes)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

In [ ]:
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

In [ ]:
y_encoded

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.float32).reshape(-1, 1)

In [ ]:
X_tensor

In [ ]:
y_tensor = y_tensor.squeeze().long()
y_tensor

In [ ]:
y_tensor.shape

In [ ]:
X.shape

In [ ]:
model = nn.Sequential(
    nn.Linear(5, 40),
    nn.ReLU(),
    nn.Linear(40, 25),
    nn.ReLU(),
    nn.Linear(25, 22))

In [ ]:
print(model)

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(), lr=0.001)

In [ ]:
n_epochs =10000
batch_size =200


for epoch in range(n_epochs):
    for i in range(0, len(X_tensor), batch_size):
        X_batch = X_tensor[i: i+batch_size]
        y_pred = model(X_batch)
        y_batch = y_tensor[i: i+batch_size]
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if epoch % 50 == 0:
        _, predicted = torch.max(y_pred, 1)
        print(f'Finished epoch {epoch}, latest loss {loss}')
        print("Predicted:", predicted[:5].tolist())
        print("Actual   :", y_batch[:5].tolist())


In [ ]:
with torch.no_grad():
    y_pred = model(X_tensor)
 
pred_classes = y_pred.argmax(dim=1)
accuracy = (pred_classes == y_tensor).float().mean()
print(f"Accuracy {accuracy*100}")

In [ ]:
import pickle

In [ ]:
import torch.onnx

dummy_input = torch.randn(1, 5)  # your input shape
torch.onnx.export(
    model,
    dummy_input,
    "finalized_model.onnx",
    input_names=['input'],
    output_names=['output'],
    dynamic_axes={'input': {0: 'batch_size'}}
)

In [ ]:
filename = 'finalized_model.pkl'
pickle.dump(model, open(filename, 'wb'))

In [ ]:
filename = 'finalized_scaler.pkl'
pickle.dump(scaler, open(filename, 'wb'))